# Natural Language Processing

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based on a TFIDF matrix and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essays written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution. If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers. They are a particularly interesting data set because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

In [ ]:
import collections # we found collections.Counter and collections.defaultdict useful
import itertools
import gzip
import re
import numpy as np
import scipy.sparse as sp

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18.  Specifically, the "pg18.txt.gz" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [ ]:
def load_federalist_corpus(filename="pg18.txt.gz", encoding='utf8'):
    """ Load the federalist papers as a tokenized list of strings"""
    with gzip.open(filename, "rt", encoding=encoding) as f:
        data = f.read()
    papers = data.split("FEDERALIST")

    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in sorted(punctuation, reverse=True):
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]

    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]

    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]

    return papers_content, authors, numbers


You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `papers` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some punctuation tokens, which have been modified to include spaces both before and after the punctuation. The `authors` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally, the `numbers` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

### Q1: Bag of words, and TFIDF

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  **Note that you need to do this manually, and not via the scikit-learn Tfidf vectorizers you used in a previous assignment question**  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

You should create the tfidf vector using numpy matrix operations and not use an existing implementation.

Important: make sure you do _not_ include the empty token `""` as one of your terms.

In [ ]:
import numpy as np
import scipy.sparse as sp
from collections import Counter
import math

def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document

    Returns: tuple: (tfidf_matrix, all_words)
        tfidf_matrix: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    # Create a list of all unique words across all documents
    all_words = sorted(set(word for doc in docs for word in doc.split() if word != ""))

    # Create a mapping from word to index
    word_to_index = {word: i for i, word in enumerate(all_words)}

    # Calculate the document frequency (DF) for each word
    doc_freq = {word: 0 for word in all_words}
    for doc in docs:
        unique_words_in_doc = set(doc.split())
        for word in unique_words_in_doc:
            if word != "":
                doc_freq[word] += 1

    # Total number of documents
    num_docs = len(docs)

    # Calculate the inverse document frequency (IDF)
    idf = {}
    for word, freq in doc_freq.items():
        if freq > 0:
            idf[word] = math.log(num_docs / freq)
        else:
            idf[word] = 0.0  # If word doesn't appear in any document, IDF is 0 (log(0) -> 0)

    # Prepare the sparse matrix components (rows, cols, values)
    rows = []
    cols = []
    values = []

    # For each document, calculate TF-IDF
    for i, doc in enumerate(docs):
        word_counts = Counter(doc.split())
        doc_len = len(doc.split())

        for word, count in word_counts.items():
            if word != "":  # Exclude empty tokens
                tf = count / doc_len
                tfidf_value = tf * idf[word]
                rows.append(i)
                cols.append(word_to_index[word])
                values.append(tfidf_value)

    # Create the sparse matrix using scipy.sparse.coo_matrix
    tfidf_matrix = sp.coo_matrix((values, (rows, cols)), shape=(num_docs, len(all_words)))

    return tfidf_matrix, all_words

Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>

### Q2: Cosine Similarity

Next, implement the following simple function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# @mugrade.local_tests
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.

    Args:
        X: sparse matrix of TFIDF scores or term frequencies

    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the
           entry M[i,j], should correspond to the cosine similarity between the
           ith and jth rows of X.
    """
    # Ensure the input is a CSR sparse matrix
    X = csr_matrix(X)

    # Compute the dot product matrix (X * X.T)
    dot_product_matrix = X.dot(X.T).toarray()

    # Compute the L2 norm for each document (row)
    norms = np.linalg.norm(X.toarray(), axis=1)

    # Normalize the dot product matrix by the product of the norms
    norms_matrix = np.outer(norms, norms)

    # Compute the cosine similarity matrix
    similarity_matrix = dot_product_matrix / norms_matrix

    # Return the similarity matrix
    return similarity_matrix

### Q3 Analyzing document authorship

Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Fill out the following function to compute and return these averages.

Hints:

1. fit a single TFIDF encoding to all papers and transform all papers using it before computing the similarity measure
2. for the cosine similarity to be useful when comparing documents, they must all be encoded the same way. Transform all papers together before comparing cosine similarity.
3. the unknown papers have author=`("HAMILTON","MADISON")`;

In [ ]:
import numpy as np
from scipy.sparse import vstack

def author_cosine_similarity(docs, authors):
    """
    Return a tuple of average cosine similarities between all the known papers for a given author
    and all the unknown papers.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
        authors: list of lists, which each list contains the author (or potential authors) of a given document

    Returns: tuple: (hamilton_mcs, madison_mcs, jay_mcs)
        hamilton_mcs: Average cosine similarity between all the known Hamilton papers and all the unknown papers.
        madison_mcs: Average cosine similarity between all the known Madison papers and all the unknown papers.
        jay_mcs: Average cosine similarity between all the known Jay papers and all the unknown papers.
    """
    # Step 1: Preprocess the documents using TF-IDF
    tfidf_matrix, all_words = tfidf(docs)

    # Step 2: Split the papers into known and unknown sets based on authorship
    hamilton_papers = []
    madison_papers = []
    jay_papers = []
    unknown_papers = []

    for i, author_list in enumerate(authors):
        if 'HAMILTON' in author_list:
            hamilton_papers.append(tfidf_matrix[i])
        elif 'MADISON' in author_list:
            madison_papers.append(tfidf_matrix[i])
        elif 'JAY' in author_list:
            jay_papers.append(tfidf_matrix[i])
        else:
            unknown_papers.append(tfidf_matrix[i])

    # Convert lists of sparse matrices to single sparse matrix using vstack
    hamilton_papers = vstack(hamilton_papers)
    madison_papers = vstack(madison_papers)
    jay_papers = vstack(jay_papers)
    unknown_papers = vstack(unknown_papers)

    # Step 3: Compute cosine similarity for each author
    hamilton_mcs = np.mean(cosine_similarity(hamilton_papers, unknown_papers))
    madison_mcs = np.mean(cosine_similarity(madison_papers, unknown_papers))
    jay_mcs = np.mean(cosine_similarity(jay_papers, unknown_papers))

    # Step 4: Return the results
    return hamilton_mcs, madison_mcs, jay_mcs

Run the local test case, `author_cosine_similarity(*load_federalist_corpus()[:2])`, to see which author has the highest degree of similarity with the unknown essays.


## N-gram language model

### Q4 Building an n-gram language model

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class, and should do it in two parts:

### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You should do this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$-th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later functions, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen. We also build a `self.vocabulary` variable, which is just a `set` object containing all the unique words across the entire set of the input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  We'll make a small tweak to the formula for perplexity in the notes because we are skipping the first $n-1$ tokens; since we only compute (#words - $(n-1)$) terms in the sum, you should divide by this denominator and not just #words). Be careful to not multiply together probabilities that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

Specifically,

\begin{equation}
\mbox{Perplexity} = 2^{\frac{-\log_2 P\left(\mathrm{word}_1, \ldots, \mathrm{word}_N\right)}{N-(n+1)}} = 2^{\text{^}}\left(\frac{-\log_2 P(\mathrm{word}_1, \ldots, \mathrm{word}_N)}{N-(n-1)}\right)
\end{equation}

where

\begin{equation}
\log_2 P(\mathrm{word}_1, \ldots, \mathrm{word}_N) = \sum_{i=n}^N \log_2 P(\mathrm{word}_i | \mathrm{word}_{i-n+1}, \ldots, \mathrm{word}_{i-1})
\end{equation}

You'll want to be careful about vocabulary sizes when it comes to the Laplace smoothing term: make sure your vocabulary size $D$ is equal to the total number of unique words that occur in either the original data used to build the language model _or_ in the text we are evaluating the perplexity of (so the size of the union of the two).

In [ ]:
class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.

        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """

        self.n = n
        self.counts = collections.defaultdict(lambda: collections.defaultdict(int))
        self.count_sums = collections.defaultdict(int)
        self.vocabulary = set()

        # Process each document
        for doc in docs:
            tokens = doc.split()  # Split the document into words
            self.vocabulary.update(tokens)

            for i in range(len(tokens) - n + 1):
                context = tuple(tokens[i:i + n - 1])  # (n-1) previous tokens
                target = tokens[i + n - 1]  # nth token

                self.counts[context][target] += 1
                self.count_sums[context] += 1  # Increment total count for this context

        self.vocabulary = set(self.vocabulary)

    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.

        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing

        Note: for the purposes of smoothing, the vocabulary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.

        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluated
                        under the model.
        """
        # Split the text into words
        tokens = text.split()
        N = len(tokens)

        # Initialize log probability sum
        log_prob_sum = 0

        # Process each word starting from the (n-1)th index
        for i in range(self.n - 1, N):
            context = tuple(tokens[i - self.n + 1:i])  # Last (n-1) words as context
            target = tokens[i]  # The nth word

            # Calculate the probability using Laplace smoothing
            context_count = self.count_sums[context]
            target_count = self.counts[context].get(target, 0)

            # Laplace smoothing: add alpha to all counts
            probability = (target_count + alpha) / (context_count + alpha * len(self.vocabulary))

            # Add the log probability to the sum (log base 2)
            log_prob_sum += math.log2(probability)

        # Calculate the perplexity
        perp = 2 ** (-log_prob_sum / (N - (self.n - 1)))
        return perp

### Q5 Author identification via language models

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`). Fill in the following function to calculate and return the mean perplexities.

In [ ]:
def mean_perplexity(docs, authors):
    """
    Evaluate the mean of the perplexity of the unknown Federalist papers for the language models
    from each of the three authors (again, using n=3 and alpha=1e-3)

    Args:
        docs: list of strings, where each string represents a space-separated document
        authors: list of lists, which each list contains the author (or potential authors) of a given document

    Returns: tuple: (perp_hamilton, perp_madison, perp_jay)
        perp_hamilton: floating point value, mean perplexity of the unknown Federalist papers for the language
                       models from Hamilton
        perp_madison: floating point value, mean perplexity of the unknown Federalist papers for the language
                       models from Madison
        perp_jay: floating point value, mean perplexity of the unknown Federalist papers for the language
                       models from Jay
    """
     # Separate known and unknown papers for each author
    hamilton_docs = []
    madison_docs = []
    jay_docs = []
    unknown_docs = []

    for doc, author in zip(docs, authors):
        if 'HAMILTON' in author:
            hamilton_docs.append(doc)
        if 'MADISON' in author:
            madison_docs.append(doc)
        if 'JAY' in author:
            jay_docs.append(doc)
        if 'HAMILTON' not in author and 'MADISON' not in author and 'JAY' not in author:
            unknown_docs.append(doc)

    # Initialize language models for each author using their known documents
    lm_hamilton = LanguageModel(hamilton_docs, n=3)
    lm_madison = LanguageModel(madison_docs, n=3)
    lm_jay = LanguageModel(jay_docs, n=3)

    # Calculate the mean perplexity for each author
    perp_hamilton = sum(lm_hamilton.perplexity(doc) for doc in unknown_docs) / len(unknown_docs)
    perp_madison = sum(lm_madison.perplexity(doc) for doc in unknown_docs) / len(unknown_docs)
    perp_jay = sum(lm_jay.perplexity(doc) for doc in unknown_docs) / len(unknown_docs)

    return perp_hamilton, perp_madison, perp_jay

Does the most likely author (i.e., the author with the smallest perplexity), match up with the author with the highest cosine similarity above?